In [1]:
import os
import tweepy as tw
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
#scientific and machine learning libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
#plotting options
import seaborn as sns
import re
import matplotlib.pyplot as plt
import missingno as ms
#% matplotlib inline
pd.set_option('display.max_colwidth',None)
import seaborn as sns
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#twint
import twint
#to avoid problems with running twint
import nest_asyncio
nest_asyncio.apply()


In [3]:
c = twint.Config()
c.Username = "viren_lathiya"
c.Custom["user"] = ["bio"]
c.User_full = True
c.Output = "users.csv"
c.Hide_output = True
c.Pandas = True
c.Stats = True
#c.Favourites = True
#twint.run.Favorites(c)
twint.run.Search(c)
Tweets_df = twint.storage.panda.Tweets_df

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [5]:
Tweets_df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,urls,photos,video,thumbnail,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,-0400,,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",[],952936045,952936045,viren_lathiya,Viren Lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],[],0,,False,0,0,0,,None,,,,,,,[],,,,
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,-0400,,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],[],952936045,952936045,viren_lathiya,Viren Lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],[],0,,False,0,0,0,,None,,,,,,,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]",,,,
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,-0400,,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],[],952936045,952936045,viren_lathiya,Viren Lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],[],0,,False,1,1,0,,None,,,,,,,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",,,,
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,-0400,,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",[],952936045,952936045,viren_lathiya,Viren Lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],[https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg],1,https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,False,7,0,3,,None,,,,,,,[],,,,
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,-0400,,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",[],952936045,952936045,viren_lathiya,Viren Lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],"[https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg, https://pbs.twimg.com/media/E6C3eVoXMAI2EmS.jpg]",1,https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,False,10,2,0,,None,,,,,,,[],,,,


In [81]:
Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [82]:
df1=Tweets_df.drop(columns=['cashtags','photos', 'video','geo',
                            'source','translate', 'trans_src',
 'trans_dest','quote_url','timezone','place','retweet','retweet_id', 
                            'retweet_date','user_rt_id','search', 'near', 'geo', 
                               'source', 'user_rt','name'])

# Clean Text in the Tweets

In [83]:
# load library
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# download the set of stop words the first time
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kalpe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [84]:
# Load stop words
stop_words = stopwords.words('english')

# Show stop words
stop_words[:5]

['i', 'me', 'my', 'myself', 'we']

In [85]:
import re 
from textblob import TextBlob 
import emoji  


import re 
from textblob import TextBlob 
import emoji  
def clean_tweet(tweet): 
    tweet = re.sub(r'@[A-Za-z0-9]+', '', str(tweet)) # remove @mentions
    tweet = re.sub(r'#', '',  str(tweet)) # remove the '#' symbol
    tweet = re.sub(r'RT[\s]+', '',  str(tweet)) # remove RT
    tweet = re.sub(r'https?\/\/S+', '',  str(tweet)) # remove the hyperlink
    tweet = re.sub(r'http\S+', '',  str(tweet)) # remove the hyperlink
    tweet = re.sub(r'www\S+', '',  str(tweet)) # remove the www
    tweet = re.sub(r'twitter+', '',  str(tweet)) # remove the twiiter
    tweet = re.sub(r'pic+', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'com', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'africa', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'innovation', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'covid-19', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'coronavirus', '',  str(tweet)) # remove the pic
    tweet = re.sub(r'technology', '',  str(tweet)) # remove the pic

    return tweet


def remove_emoji(tweet):
    return emoji.get_emoji_regexp().sub(u'', tweet)



In [86]:
df1['cleaned_text']=df1['tweet'].apply(clean_tweet)
df1['cleaned_text']=df1['cleaned_text'].apply(remove_emoji)

In [87]:
# Remove stop words
df1['cleaned_text'] = df1['cleaned_text'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop_words))

In [88]:
df1.head()

,id,conversation_id,created_at,date,tweet,language,hashtags,user_id,user_id_str,username,day,hour,link,urls,thumbnail,nlikes,nreplies,nretweets,reply_to,cleaned_text
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",952936045,952936045,viren_lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],,0,0,0,[],check latest article: the future influencer marketing is here via influencermarketing b413 mygbc
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],952936045,952936045,viren_lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],,0,0,0,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]","_bungee this _bungee gatineau, quebec. it best experience life"
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],952936045,952936045,viren_lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],,1,1,0,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",_montas i'm fully vaccinated also started traveling ️ free
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",952936045,952936045,viren_lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,7,0,3,[],my top 5 marketing analytics tools: 1. - analyze your campaign with deeper data 2. - s.e.o analytics tool 3. - behavioral analytics tool 4. - heat map websites 5. - content/social analysis mark1051 knowyoursocial ai
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",952936045,952936045,viren_lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,10,2,0,[],"3....2....1 jump, adrenaline flowing !! just canada's highest bungee jump 200 ft high. rebounce 160ft. boat ride back thinking crazy it. sky dive next mark1051 bungee weekend"


# Sentiment Modelling 

In [89]:
import numpy as np
import string
import re
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [90]:
# get functionality of subjectivity and polarity 
def getSubjectivity(text):
    return TextBlob( str(text)).sentiment.subjectivity

def getPolarity(text):
    return TextBlob( str(text)).sentiment.polarity

In [91]:
df1.dropna(subset=['cleaned_text'], inplace = True)
df1.reset_index(drop=True, inplace=True)
df1.head()

,id,conversation_id,created_at,date,tweet,language,hashtags,user_id,user_id_str,username,day,hour,link,urls,thumbnail,nlikes,nreplies,nretweets,reply_to,cleaned_text
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",952936045,952936045,viren_lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],,0,0,0,[],check latest article: the future influencer marketing is here via influencermarketing b413 mygbc
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],952936045,952936045,viren_lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],,0,0,0,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]","_bungee this _bungee gatineau, quebec. it best experience life"
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],952936045,952936045,viren_lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],,1,1,0,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",_montas i'm fully vaccinated also started traveling ️ free
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",952936045,952936045,viren_lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,7,0,3,[],my top 5 marketing analytics tools: 1. - analyze your campaign with deeper data 2. - s.e.o analytics tool 3. - behavioral analytics tool 4. - heat map websites 5. - content/social analysis mark1051 knowyoursocial ai
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",952936045,952936045,viren_lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,10,2,0,[],"3....2....1 jump, adrenaline flowing !! just canada's highest bungee jump 200 ft high. rebounce 160ft. boat ride back thinking crazy it. sky dive next mark1051 bungee weekend"


In [92]:
df1['Subjectivity'] = df1['cleaned_text'].apply(getSubjectivity)
df1['Polarity'] = df1['cleaned_text'].apply(getPolarity)
df1.head()

,id,conversation_id,created_at,date,tweet,language,hashtags,user_id,user_id_str,username,day,hour,link,urls,thumbnail,nlikes,nreplies,nretweets,reply_to,cleaned_text,Subjectivity,Polarity
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",952936045,952936045,viren_lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],,0,0,0,[],check latest article: the future influencer marketing is here via influencermarketing b413 mygbc,0.5125,0.25
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],952936045,952936045,viren_lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],,0,0,0,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]","_bungee this _bungee gatineau, quebec. it best experience life",0.3000,1.00
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],952936045,952936045,viren_lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],,1,1,0,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",_montas i'm fully vaccinated also started traveling ️ free,0.8000,0.40
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",952936045,952936045,viren_lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,7,0,3,[],my top 5 marketing analytics tools: 1. - analyze your campaign with deeper data 2. - s.e.o analytics tool 3. - behavioral analytics tool 4. - heat map websites 5. - content/social analysis mark1051 knowyoursocial ai,0.5000,0.50
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",952936045,952936045,viren_lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,10,2,0,[],"3....2....1 jump, adrenaline flowing !! just canada's highest bungee jump 200 ft high. rebounce 160ft. boat ride back thinking crazy it. sky dive next mark1051 bungee weekend",0.3600,-0.11


In [93]:
def get_Polarity_Analysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'
def get_Subjectivity_Analysis(score):
    if score >  0:
      return 'Opinion'
    else:
      return 'Fact'

df1['Analysis_Polarity'] = df1['Polarity'].apply(get_Polarity_Analysis)

df1['Analysis_Subjectivity'] = df1['Subjectivity'].apply(get_Subjectivity_Analysis)

# Show the dataframe
df1.head(20)



,id,conversation_id,created_at,date,tweet,language,hashtags,user_id,user_id_str,username,day,hour,link,urls,thumbnail,nlikes,nreplies,nretweets,reply_to,cleaned_text,Subjectivity,Polarity,Analysis_Polarity,Analysis_Subjectivity
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",952936045,952936045,viren_lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],,0,0,0,[],check latest article: the future influencer marketing is here via influencermarketing b413 mygbc,0.512500,0.250000,Positive,Opinion
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],952936045,952936045,viren_lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],,0,0,0,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]","_bungee this _bungee gatineau, quebec. it best experience life",0.300000,1.000000,Positive,Opinion
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],952936045,952936045,viren_lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],,1,1,0,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",_montas i'm fully vaccinated also started traveling ️ free,0.800000,0.400000,Positive,Opinion
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",952936045,952936045,viren_lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,7,0,3,[],my top 5 marketing analytics tools: 1. - analyze your campaign with deeper data 2. - s.e.o analytics tool 3. - behavioral analytics tool 4. - heat map websites 5. - content/social analysis mark1051 knowyoursocial ai,0.500000,0.500000,Positive,Opinion
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",952936045,952936045,viren_lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,10,2,0,[],"3....2....1 jump, adrenaline flowing !! just canada's highest bungee jump 200 ft high. rebounce 160ft. boat ride back thinking crazy it. sky dive next mark1051 bungee weekend",0.360000,-0.110000,Negative,Opinion
5,1413261815375667205,1413261815375667205,1.625783e+12,2021-07-08 18:20:48,My Top 5 Search Engine Optimization Tools: 1. @Moz - For Smarter Marketing 2. @ubersuggest_seo - Generating New Keyword Ideas 3. @semrush - For Competitive Research 4. @mangools_com - SERP Checker 5. @ahrefs - Biggest Index Of Live Backlinks #MARK1051 #KnowYourSocial #SEO https://t.co/blnNMFEh7J,en,"[mark1051, knowyoursocial, seo]",952936045,952936045,viren_lathiya,4,18,https://twitter.com/viren_lathiya/status/1413261815375667205,[],https://pbs.twimg.com/media/E5zpuhgWYAEsYUa.png,4,0,0,[],my top 5 search engine optimization tools: 1. - for smarter marketing 2. _seo -

In [136]:
df1['id']=df1['id'].astype(str)

In [137]:
df1['conversation_id']=df1['conversation_id'].astype(str)

In [138]:
df1.head(10)

,id,conversation_id,created_at,date,tweet,language,hashtags,user_id,user_id_str,username,day,hour,link,urls,thumbnail,nlikes,nreplies,nretweets,reply_to,cleaned_text,Subjectivity,Polarity,Analysis_Polarity,Analysis_Subjectivity,replied
0,1417566841225060357,1417566841225060357,1.626809e+12,2021-07-20 15:27:26,Check out my latest article: The Future of Influencer Marketing Is Here https://t.co/pH4yx4VKbh via @LinkedIn #InfluencerMarketing #B413 #MyGBC,en,"[influencermarketing, b413, mygbc]",952936045,952936045,viren_lathiya,2,15,https://twitter.com/viren_lathiya/status/1417566841225060357,[https://www.linkedin.com/pulse/future-influencer-marketing-here-virenkumar-lathiya],,0,0,0,[],check latest article: the future influencer marketing is here via influencermarketing b413 mygbc,0.512500,0.250000,Positive,Opinion,1
1,1415377271616122883,1414332023267860487,1.626287e+12,2021-07-14 14:26:52,"@AnushaS91 @GCB_Bungee This is @GCB_Bungee in Gatineau, Quebec. It was the best experience of my life 🔥",en,[],952936045,952936045,viren_lathiya,3,14,https://twitter.com/viren_lathiya/status/1415377271616122883,[],,0,0,0,"[{'screen_name': 'AnushaS91', 'name': 'Anusha Subramanian', 'id': '1347988517104873473'}, {'screen_name': 'GCB_Bungee', 'name': 'GreatCanadianBungee', 'id': '1262706086'}]","_bungee this _bungee gatineau, quebec. it best experience life",0.300000,1.000000,Positive,Opinion,1
2,1414950907813453833,1414691468430745601,1.626186e+12,2021-07-13 10:12:39,@adi_montas I'm fully vaccinated and also started traveling 🏖️ #free,en,[free],952936045,952936045,viren_lathiya,2,10,https://twitter.com/viren_lathiya/status/1414950907813453833,[],,1,1,0,"[{'screen_name': 'adi_montas', 'name': 'Adi Montas', 'id': '130414637'}]",_montas i'm fully vaccinated also started traveling ️ free,0.800000,0.400000,Positive,Opinion,1
3,1414945669035528209,1414945669035528209,1.626184e+12,2021-07-13 09:51:50,My Top 5 Marketing Analytics Tools: 1. @googleanalytics - Analyze Your Campaign With Deeper Data 2. @Moz - S.E.O Analytics Tool 3. @Kissmetrics - Behavioral Analytics Tool 4. @CrazyEgg - Heat Map on Websites 5. @BuzzSumo - Content/Social Analysis #MARK1051 #KnowYourSocial #AI https://t.co/wUM9PetYMh,en,"[mark1051, knowyoursocial, ai]",952936045,952936045,viren_lathiya,2,09,https://twitter.com/viren_lathiya/status/1414945669035528209,[],https://pbs.twimg.com/media/E6LllneXIAUbHyt.jpg,7,0,3,[],my top 5 marketing analytics tools: 1. - analyze your campaign with deeper data 2. - s.e.o analytics tool 3. - behavioral analytics tool 4. - heat map websites 5. - content/social analysis mark1051 knowyoursocial ai,0.500000,0.500000,Positive,Opinion,1
4,1414332023267860487,1414332023267860487,1.626038e+12,2021-07-11 17:13:26,"3....2....1 Jump, Adrenaline is Flowing !! 🔥 Just did Canada's highest bungee jump 200 ft high. Rebounce 160ft. and a boat ride back thinking how crazy was it. Sky Dive Next 🔜 #MARK1051 #bungee #Weekend https://t.co/CIOibw4Zio",en,"[mark1051, bungee, weekend]",952936045,952936045,viren_lathiya,7,17,https://twitter.com/viren_lathiya/status/1414332023267860487,[],https://pbs.twimg.com/media/E6C3ds1XMAAn09R.jpg,10,2,0,[],"3....2....1 jump, adrenaline flowing !! just canada's highest bungee jump 200 ft high. rebounce 160ft. boat ride back thinking crazy it. sky dive next mark1051 bungee weekend",0.360000,-0.110000,Negative,Opinion,1
5,1413261815375667205,1413261815375667205,1.625783e+12,2021-07-08 18:20:48,My Top 5 Search Engine Optimization Tools: 1. @Moz - For Smarter Marketing 2. @ubersuggest_seo - Generating New Keyword Ideas 3. @semrush - For Competitive Research 4. @mangools_com - SERP Checker 5. @ahrefs - Biggest Index Of Live Backlinks #MARK1051 #KnowYourSocial #SEO https://t.co/blnNMFEh7J,en,"[mark1051, knowyoursocial, seo]",952936045,952936045,viren_lathiya,4,18,https://twitter.com/viren_lathiya/status/1413261815375667205,[],https://pbs.twimg.com/media/E5zpuhgWYAEsYUa.png,4,0,0,[],my top 5 search engine optimization tools: 1. - for smarter m

In [139]:
main_df=df1

In [141]:
main_df.to_csv("Main_tweets.csv")